***Imports***

In [ ]:
# imports
import cv2
import os
import numpy as np
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.transforms.functional  import InterpolationMode
import torchvision
import torch
import torch
import torch.nn as nn
from torchsummary import summary
from statistics import mean
import PIL
import matplotlib.pyplot as plt
import numpy as np

# **Handle CycleGan Import**

In [ ]:
# Clone the CycleGAN repository
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
# Navigate to the cloned repository
%cd pytorch-CycleGAN-and-pix2pix


In [ ]:
# Install necessary Python packages
!pip install -r requirements.txt


mount drive and upload if needed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# dataset upload (Now you have a dataset and need to upload it for training)

# download dataset
from google.colab import files

files.upload()

In [ ]:
!cp -r /content/pytorch-CycleGAN-and-pix2pix/WIN_20240627_21_19_57_Pro.mp4 /content/drive/MyDrive/Check_cycleGan

train

In [ ]:
!python train.py --dataroot /content/drive/MyDrive/dataset \
--name your_dataset_cyclegan3 \
--model cycle_gan \
--n_epochs 30 \
--n_epochs_decay 120 \
--save_epoch_freq 10 \
--lr 0.0002 --batch_size 1 \
--ngf 64 --ndf 64 \
--checkpoints_dir /content/drive/MyDrive/cyclegan_checkpoints \
--epoch_count 125


In [ ]:
from IPython.display import Image, display
import glob
import os

# Path to the directory containing generated images
# Ensure this path is correct and points to your training images
result_dir = '/content/drive/MyDrive/cyclegan_checkpoints/your_dataset_cyclegan3/web/images'

# Check if the directory exists and list its contents
if not os.path.exists(result_dir):
    print(f"Directory {result_dir} does not exist.")
else:
    print(f"Contents of {result_dir}:")
    !ls {result_dir}

# Display all images in the directory
for img_path in glob.glob(f"{result_dir}/*.*"):  # Change the extension as needed
    image_name = os.path.basename(img_path)
    # Print and display only images containing 'fake' or 'real' in the filename
    if 'fake' in image_name or 'real' in image_name:
        print(f"Displaying image: {image_name}")
        display(Image(filename=img_path))


train 2.0

test

In [ ]:
!python test.py --dataroot /content/drive/MyDrive/dataset/testA \
--name your_dataset_cyclegan3 \
--model test \
--results_dir /content/drive/MyDrive/cycleGAN_res1 \
--no_dropout \
--gpu_ids -1 \
--checkpoints_dir /content/drive/MyDrive/cyclegan_checkpoints \
--epoch latest_net_G_B \
--direction AtoB



show test

In [ ]:
from IPython.display import Image, display
import glob

# Path to the directory containing generated images
result_dir = '/content/drive/MyDrive/cycleGAN_res/your_dataset_cyclegan_pretrained/test_latest/images'

# Display all images in the directory
for img_path in glob.glob(f"{result_dir}/*.png"):
    display(Image(filename=img_path))

In [ ]:
import cv2
import os
import subprocess

# Function to extract frames from the video
def extract_frames(video_path, output_dir):
    """Extracts frames from a video and saves them as individual images."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    video_capture = cv2.VideoCapture(video_path)
    success, frame = video_capture.read()
    count = 0

    while success:
        frame_path = f"{output_dir}/frame_{count:04d}.jpg"
        cv2.imwrite(frame_path, frame)
        success, frame = video_capture.read()
        count += 1

    video_capture.release()
    print(f"Extracted {count} frames from the video to {output_dir}.")

# Function to process each frame using test.py
def process_frames(input_dir, output_dir):
    """Processes frames in the input directory using test.py and saves the results."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    command = f"python test.py --dataroot {input_dir} --results_dir {output_dir} --name experiment_name --model test --no_dropout --gpu_ids -1"
    print(f"Running command: {command}")
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    print("Command output:", result.stdout)
    print("Command error (if any):", result.stderr)

    output_files = os.listdir(output_dir)
    if not output_files:
        print(f"No output frames found in {output_dir}. Please check test.py configuration and execution.")
    else:
        print(f"Processed frames found in {output_dir}. Total: {len(output_files)} files.")

# Function to create a video from processed frames
def create_video_from_frames(frames_dir, output_video_path, frame_rate=30):
    """Creates a video from a sequence of image frames."""
    frame_files = [f for f in os.listdir(frames_dir) if f.endswith('.jpg')]
    frame_files.sort()  # Ensure frames are in order

    if not frame_files:
        print(f"No frames found in the directory {frames_dir}.")
        return

    print(f"Found {len(frame_files)} frames in {frames_dir} for video creation.")

    first_frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
    height, width, layers = first_frame.shape
    size = (width, height)

    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, size)

    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(frames_dir, frame_file))
        video_writer.write(frame)

    video_writer.release()
    print(f"Video saved as {output_video_path}")

# Main function to run the entire process
def main(video_path, frames_dir, processed_frames_dir, output_video):
    """Main function to handle the entire process of extracting, processing, and creating video."""
    extract_frames(video_path, frames_dir)
    process_frames(frames_dir, processed_frames_dir)
    create_video_from_frames(processed_frames_dir, output_video)

# Parameters
video_path = '/content/drive/MyDrive/Check_cycleGan/WIN_20240627_21_19_57_Pro.mp4'
frames_dir = '/content/drive/MyDrive/Check_cycleGan/frames'
processed_frames_dir = '/content/drive/MyDrive/Check_cycleGan/output_frames'
output_video = '/content/drive/MyDrive/Check_cycleGan/output_video.mp4'

# Run the main function
main(video_path, frames_dir, processed_frames_dir, output_video)


In [ ]:
!python test.py --dataroot /content/drive/MyDrive/Check_cycleGan/frames --results_dir /content/drive/MyDrive/Check_cycleGan/output_frames --name experiment_name --model test --no_dropout --gpu_ids -1


In [ ]:
from IPython.display import clear_output
import time
import cv2
from google.colab.patches import cv2_imshow
import torch
from models import create_model
from data import create_dataset
from options.test_options import TestOptions
from util import util
from PIL import Image
import numpy as np
from torchsummary import summary
from torchvision import transforms
from data.base_dataset import BaseDataset, get_transform
import matplotlib.pyplot as plt
import os

# Initialize TestOptions
opt = TestOptions()
opt.dataroot = '/content/drive/MyDrive/dataset'
opt.name = 'your_dataset_cyclegan3'
opt.model = 'test'
opt.checkpoints_dir = '/content/drive/MyDrive/cyclegan_checkpoints'
opt.no_dropout = True
opt.gpu_ids = []
opt.isTrain = False
opt.input_nc = 3
opt.output_nc = 3
opt.ngf = 64
opt.ndf = 64
opt.netD = 'basic'
opt.netG = 'resnet_9blocks'
opt.n_layers_D = 3
opt.norm = 'instance'
opt.init_type = 'normal'
opt.init_gain = 0.02
opt.dataset_mode = 'unaligned'
opt.direction = 'AtoB'
opt.load_size = 286
opt.crop_size = 256
opt.max_dataset_size = float("inf")
opt.preprocess = 'resize_and_crop'
opt.display_winsize = 256
opt.epoch = 'latest'
opt.load_iter = 0
opt.verbose = False
opt.suffix = ''
opt.model_suffix = ''
opt.use_wandb = False
opt.wandb_project_name = 'CycleGAN-and-pix2pix'
opt.results_dir = './results/'
opt.aspect_ratio = 1.0
opt.phase = 'test'
opt.eval = False
opt.num_test = 50
opt.num_threads = 0   # test code only supports num_threads = 0
opt.batch_size = 1    # test code only supports batch_size = 1
opt.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
opt.no_flip = True    # no flip; comment this line if results on flipped images are needed.
opt.display_id = -1

# Create dataset
dataset = create_dataset(opt)
opt.load_path = '/content/drive/MyDrive/cyclegan_checkpoints/your_dataset_cyclegan3/latest_net_G_A.pth'
# Initialize model
model = create_model(opt)
model.setup(opt)
model.eval()

# Function to preprocess an image
def preprocess(image, opt):
    if image.ndim == 2 or image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    elif image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    pil_image = Image.fromarray(image)
    transform = get_transform(opt, grayscale=False)
    image_tensor = transform(pil_image).unsqueeze(0)
    return image_tensor

# Function to perform inference
def infer(image_tensor, opt, model):
    data = {'A': image_tensor, 'A_paths': ''}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()

    fake_image = visuals['fake'].squeeze().cpu().numpy().transpose(1, 2, 0)
    fake_image = ((fake_image - fake_image.min()) / (fake_image.max() - fake_image.min()) * 255).astype(np.uint8)
    fake_image = cv2.cvtColor(fake_image, cv2.COLOR_BGR2RGB)

    return fake_image
def video_stream():
  js = Javascript('
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame(width, height) {
      if (!shutdown) {
         window.requestAnimationFrame(function() {
            onAnimationFrame(width, height);
    });
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, width, height);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

# Initialize video streaming
video_stream()

# Main loop to capture and process frames
while True:
    js_reply = video_frame('', '', width, height)
    if not js_reply:
        break

    frame = js_to_image(js_reply['img'])
    frame_tensor = preprocess(frame, opt)
    inferred_image = infer(frame_tensor, opt, model)

    clear_output(wait=True)
    cv2_imshow(inferred_image)

video_stream_stop()


In [ ]:

import PIL
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import io
# Place your code here (2): define your own video width and height
width = 160
height = 120

# functions for capturing webcam
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes


def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame(width, height) {
      if (!shutdown) {
         window.requestAnimationFrame(function() {
            onAnimationFrame(width, height);
    });
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, width, height);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom(width, height) {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = width; //video.videoWidth;
      captureCanvas.height = height; //video.videoHeight;
      window.requestAnimationFrame(function() {
      onAnimationFrame(width, height);
    });
      return stream;
    }

 async function stop_sream() {
        shutdown = true;
        removeDom();

        return '';
 }

    async function stream_frame(label, imgData, width, height) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom(width, height);

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox, width, videoHeight):
  data = eval_js('stream_frame("{}", "{}", "{}", "{}")'.format(label, bbox, width, height))
  return data

def video_stream_stop():
  eval_js('stop_sream("{}")')
import cv2
from google.colab.patches import cv2_imshow
import torch
from models import create_model
from data import create_dataset
from options.test_options import TestOptions
from util import util
from PIL import Image
import numpy as np
from torchsummary import summary
from torchvision import transforms
from data.base_dataset import BaseDataset, get_transform
import matplotlib.pyplot as plt
import os
def display_images_in_row(images):
    images_rgb = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in images]
    plt.figure(figsize=(15, 5))
    for i, image in enumerate(images_rgb):
        plt.subplot(1, len(images_rgb), i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()
    plt.show()
def preprocess(image, opt):
    """
    Preprocess an image to match the tensor format produced by the dataset iterator.

    Args:
        image (numpy.ndarray): The input image in numpy array format.
        opt (Option class): The options class with the same parameters used for training.

    Returns:
        torch.Tensor: The preprocessed image tensor.
    """
    # Convert the image to RGB if it is not already in RGB format
    if image.ndim == 2 or image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    elif image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert image to PIL format for torchvision transforms
    pil_image = PIL.Image.fromarray(image)

    # Get the transformation pipeline
    transform_pipeline = get_transform(opt, grayscale=False)

    # Apply the transformation pipeline
    image_tensor = transform_pipeline(pil_image)

    # Add a batch dimension
    image_tensor = image_tensor.unsqueeze(0)

    return image_tensor
# Create a new TestOptions object
opt = TestOptions()

# Manually set all required options
opt.dataroot = '/content/drive/MyDrive/dataset'
opt.name = 'your_dataset_cyclegan3'
opt.model = 'test'
opt.checkpoints_dir = '/content/drive/MyDrive/cyclegan_checkpoints'  # Use local directory
opt.no_dropout = True
opt.gpu_ids = []
opt.isTrain = False


opt.input_nc = 3
opt.output_nc = 3
opt.ngf = 64
opt.ndf = 64
opt.netD = 'basic'
opt.netG = 'resnet_9blocks'
opt.n_layers_D = 3
opt.norm = 'instance'
opt.init_type = 'normal'
opt.init_gain = 0.02
opt.dataset_mode = 'unaligned'
opt.direction = 'AtoB'
opt.load_size = 286
opt.crop_size = 256
opt.max_dataset_size = float("inf")
opt.preprocess = 'resize_and_crop'
opt.display_winsize = 256
opt.epoch = 'latest'
opt.load_iter = 0
opt.verbose = False
opt.suffix = ''
opt.model_suffix = ''
opt.use_wandb = False
opt.wandb_project_name = 'CycleGAN-and-pix2pix'
opt.results_dir = './results/'
opt.aspect_ratio = 1.0
opt.phase = 'test'
opt.eval = False
opt.num_test = 50


# get test options
# hard-code some parameters for test
opt.num_threads = 0   # test code only supports num_threads = 0
opt.batch_size = 1    # test code only supports batch_size = 1
opt.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
opt.no_flip = True    # no flip; comment this line if results on flipped images are needed.
opt.display_id = -1
dataset = create_dataset(opt)  # create a dataset given opt.dataset_mode and other options

# Initialize the model
model = create_model(opt)
model.setup(opt)
model.eval()

def infer(image_tensor, opt, model):
  data = {'A': image_tensor,'A_paths': ''}
  model.set_input(data)
  model.test()
  visuals = model.get_current_visuals()

  real_image = visuals['real']
  real_image_np = real_image.squeeze().cpu().numpy().transpose(1, 2, 0)
  real_image_np = ((real_image_np - real_image_np.min()) / (real_image_np.max() - real_image_np.min()) * 255).astype(np.uint8)
  real_image_np = cv2.cvtColor(real_image_np, cv2.COLOR_BGR2RGB)

  fake_image = visuals['fake']
  fake_image_np = fake_image.squeeze().cpu().numpy().transpose(1, 2, 0)
  fake_image_np = ((fake_image_np - fake_image_np.min()) / (fake_image_np.max() - fake_image_np.min()) * 255).astype(np.uint8)
  fake_image_np = cv2.cvtColor(fake_image_np, cv2.COLOR_BGR2RGB)

  return fake_image_np, real_image_np

from IPython.display import clear_output
import time
# Start the video stream
video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox, width, height)
    if not js_reply:
        break

    # Process the frame
    frame = js_to_image(js_reply['img'])
    frame_tensor = preprocess(frame, opt)
    '''data = {'A': frame_tensor,'A_paths': ''}
    model.set_input(data)
    model.test()
    visuals = model.get_current_visuals()
    output_image = visuals['fake']

    output_image_np = output_image.squeeze().cpu().numpy().transpose(1, 2, 0)
    output_image_np = ((output_image_np - output_image_np.min()) / (output_image_np.max() - output_image_np.min()) * 255).astype(np.uint8)
    output_image_np = cv2.cvtColor(output_image_np, cv2.COLOR_BGR2RGB)'''
    inferred_image, real_image = infer(frame_tensor,opt, model)
    # Display the output image
    clear_output(wait=True)
    cv2_imshow(inferred_image)
    #time.sleep(0.25)

video_stream_stop()